In [1]:
import ParticleFilters: obs_weight
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP, POMDPPolicies
using POMDPToolbox
using Distributions
using Parameters
using BeliefUpdaters
using Plots
using StaticArrays

┌ Warning: Importing POMDPToolbox. As of Julia 0.7 POMDPToolbox has been discontinued. Please use POMDPModelTools, POMDPPolicies, BeliefUpdaters, and/or POMDPSimulators instead.
└ @ POMDPToolbox C:\Users\cveda\.julia\packages\POMDPToolbox\OdZy7\src\POMDPToolbox.jl:4
┌ Warning: Error requiring `UnicodePlots` from `Plots`
│   exception = (LoadError("C:\\Users\\cveda\\.julia\\packages\\Plots\\CBYO0\\src\\backends\\unicodeplots.jl", 77, UndefVarError(:UserColorType)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00000000026f4a81, Ptr{Nothing} @0x000000000278282b, Ptr{Nothing} @0x000000000eb69de7, Ptr{Nothing} @0x0000000002781afc, Ptr{Nothing} @0x00000000027816f4, Ptr{Nothing} @0x0000000002782048, Ptr{Nothing} @0x0000000002782c85, Base.InterpreterIP in top-level CodeInfo for Plots at statement 3, Ptr{Nothing} @0x000000000279f287, Ptr{Nothing} @0x000000000279fdc1, Ptr{Nothing} @0x00000000027a0d0f, Ptr{Nothing} @0x000000000f144dfb, Ptr{Nothing} @0x000000000f144a1f, Ptr{Nothing} @0x

In [2]:
@with_kw struct RacecarParameters
    # Rewards
    r_crash::Real = -100 #Terminal State
    r_avoided_stop::Real = -4 #Terminal State
    r_avoided_continue::Real = 20 # No Crash yet by continuing straight.
    r_avoided_left::Real = -5 #Terminal State
    r_avoided_right::Real = -6 #Terminal State
    
    # Transition probability
    p_threat_arises::Real = .15   #Probability that the car behind is going to be a threat
    p_crash_when_continue_wthreat::Real = .5
    #Obervation probabilities
    p_collision_detected::Real = .7 # Suboptimal conditions when the lidar only has a 70% chance of detecting the collision
    p_false_alarm::Real = .1        # when no threat
end

RacecarParameters

In [3]:
params = RacecarParameters();

In [4]:
begin
    @enum State Crash Collision_AC Collision_AS Collision_AL Collision_AR 
    @enum Action Continue Stop Left_T Right_T 
    @enum Observation Collision_D Collision_U
end

In [5]:
𝒮 = [Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR]

5-element Vector{State}:
 Crash::State = 0
 Collision_AC::State = 1
 Collision_AS::State = 2
 Collision_AL::State = 3
 Collision_AR::State = 4

In [6]:
𝒜 = [Continue, Stop, Left_T, Right_T]

4-element Vector{Action}:
 Continue::Action = 0
 Stop::Action = 1
 Left_T::Action = 2
 Right_T::Action = 3

In [7]:
𝒪 = [Collision_D, Collision_U]

2-element Vector{Observation}:
 Collision_D::Observation = 0
 Collision_U::Observation = 1

In [8]:
initialstate_distr = Deterministic(Collision_AC)

Deterministic{State}(Collision_AC)

In [9]:
function T(s::State, a::Action)
    p_Crash_I::Real = params.p_crash_when_continue_wthreat
    p_Threat::Real = params.p_threat_arises
    p_Crash::Real = p_Threat*p_Crash_I
    if a == Right_T
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,0,0,1])
    elseif a == Left_T
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,0,1,0]) 
    elseif a == Stop
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,1,0,0])
    elseif s == Collision_AC  && a == Continue
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [p_Crash,1-p_Crash,0,0])
    end
end

T (generic function with 1 method)

In [10]:
function O(s::State, a::Action, sp::State)
    p_True_Detect = params.p_collision_detected
    p_False_Detect = params.p_false_alarm
    if sp == Crash
        return SparseCat([Collision_D, Collision_U], [p_True_Detect, 1-p_True_Detect])
    elseif sp == Collision_AC
        return SparseCat([Collision_D, Collision_U], [p_False_Detect, 1-p_False_Detect])
    end
end        

O (generic function with 1 method)

In [11]:
function R(s::State, a::Action)
    return(s == Crash ? params.r_crash : 0) + (a == Continue ? params.r_avoided_continue : 0) + 
                                              (a == Stop ? params.r_avoided_stop : 0) +
                                              (a == Left_T ? params.r_avoided_left : 0) + 
                                              (a == Right_T ? params.r_avoided_right : 0) 
end

R (generic function with 1 method)

In [12]:
# R(Collision_AC, Continue)

In [13]:
γ = 0.9

0.9

In [14]:
abstract type Racecar <: POMDP{State, Action, Observation}end

In [15]:
pomdp = QuickPOMDP(Racecar,
    states = 𝒮,
    actions = 𝒜,
    observations = 𝒪,
    transition = T,
    reward = R,
    observation = O,
    discount = γ,
    initialstate = initialstate_distr);

In [16]:
struct ActWhenThreat <: Policy end

In [17]:
struct ActWhenBelievedThreat <: Policy end

In [18]:
const Belief = Vector{Real};

In [19]:
function POMDPs.action(::ActWhenThreat, o::Observation)
    return o == Collision_D ? Stop : Continue
end;

In [20]:
function POMDPs.action(::ActWhenBelievedThreat, b::Belief)
    if b[1] > b[2]
        return Stop
    else
        return Continue
    end
end;

In [21]:
updater(pomdp::QuickPOMDP{Racecar}) = DiscreteUpdater(pomdp)

updater (generic function with 1 method)

In [22]:
b0 = uniform_belief(pomdp); b0.b

5-element Vector{Float64}:
 0.2
 0.2
 0.2
 0.2
 0.2

In [23]:
begin
    a1 = Continue 
    o1 = Collision_U
    b1 = update(DiscreteUpdater(pomdp), b0, a1, o1)
    b1.b
end

LoadError: MethodError: no method matching support(::Nothing)
[0mClosest candidates are:
[0m  support([91m::ParticleFilters.ParticleCollection[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:103
[0m  support([91m::ParticleFilters.AbstractParticleBelief[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:173
[0m  support([91m::DiscreteBelief[39m) at C:\Users\cveda\.julia\packages\BeliefUpdaters\gFL9O\src\discrete.jl:73
[0m  ...

In [24]:
using QMDP

In [25]:
solver = QMDPSolver()

QMDPSolver{DiscreteValueIteration.ValueIterationSolver}(DiscreteValueIteration.ValueIterationSolver(100, 0.001, false, true, Float64[]))

In [26]:
policy = solve(solver, pomdp)

INFO: POMDPLinter requirements for solve(::ValueIterationSolver, ::Union{MDP,POMDP}) and dependencies. ([✔] = implemented correctly; [X] = not implemented; [?] = could not determine)

For solve(::ValueIterationSolver, ::Union{MDP,POMDP}):
  [✔] discount(::UnderlyingMDP{QuickPOMDP{Racecar, State, Action, Observation, NamedTuple{(:stateindex, :isterminal, :obsindex, :states, :observations, :discount, :actions, :observation, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, Bool, Dict{Observation, Int64}, Vector{State}, Vector{Observation}, Float64, Vector{Action}, typeof(O), Dict{Action, Int64}, typeof(T), typeof(R), Deterministic{State}}}}StateAction})
  [✔] transition(::UnderlyingMDP{QuickPOMDP{Racecar, State, Action, Observation, NamedTuple{(:stateindex, :isterminal, :obsindex, :states, :observations, :discount, :actions, :observation, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, Bool, Dict{Observation, Int64}, Vector{State}

LoadError: MethodError: no method matching support(::Nothing)
[0mClosest candidates are:
[0m  support([91m::ParticleFilters.ParticleCollection[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:103
[0m  support([91m::ParticleFilters.AbstractParticleBelief[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:173
[0m  support([91m::DiscreteBelief[39m) at C:\Users\cveda\.julia\packages\BeliefUpdaters\gFL9O\src\discrete.jl:73
[0m  ...

In [27]:
using FIB

In [28]:
fib_solver = FIBSolver()

FIBSolver(100, 0.001, false)

In [29]:
fib_policy = solve(fib_solver, pomdp)

LoadError: MethodError: no method matching support(::Nothing)
[0mClosest candidates are:
[0m  support([91m::ParticleFilters.ParticleCollection[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:103
[0m  support([91m::ParticleFilters.AbstractParticleBelief[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:173
[0m  support([91m::DiscreteBelief[39m) at C:\Users\cveda\.julia\packages\BeliefUpdaters\gFL9O\src\discrete.jl:73
[0m  ...

In [30]:
using Plots; default(fontfamily="Computer Modern", framestyle=:box) # LaTex-style

In [31]:
action(policy, b)

LoadError: UndefVarError: policy not defined

In [32]:
using BasicPOMCP

In [33]:
pomcp_solver = POMCPSolver()

POMCPSolver
  max_depth: Int64 20
  c: Float64 1.0
  tree_queries: Int64 1000
  max_time: Float64 Inf
  tree_in_info: Bool false
  default_action: ExceptionRethrow ExceptionRethrow()
  rng: Random._GLOBAL_RNG Random._GLOBAL_RNG()
  estimate_value: RolloutEstimator


In [34]:
pomcp_planner = solve(pomcp_solver, pomdp);

In [35]:
initialstate(pomdp)

Deterministic{State}(Collision_AC)

In [36]:
using D3Trees
b = initial_state_distribution(p)
a, i = action_info(planner, b)
D3Tree(i[:tree], init_expand=2)

LoadError: UndefVarError: initial_state_distribution not defined

In [37]:
using D3Trees

In [38]:
tree = D3Tree(info[:tree], init_expand=3)

LoadError: UndefVarError: info not defined